## <center> Projet 3 - Parcours Machine Learning :<br/>Consomation électrique de la ville de Seattle </center>

<p style="text-align:center;">Vous travaillez pour la ville de Seattle. Pour atteindre son objectif de ville neutre en émissions de carbone en 2050, votre équipe s’intéresse de près aux émissions des bâtiments non destinés à l’habitation.</p>

<center><img src="image/logo_seattle.png" alt="logo_seattle" width="500" /></center>

##### Les données

Les données de consommation sont [à télécharger ici](https://www.kaggle.com/city-of-seattle/sea-building-energy-benchmarking#2015-building-energy-benchmarking.csv).

##### Problématique de la ville de Seattle

<p style="text-align:justify;">Des relevés minutieux ont été effectués par vos agents en 2015 et en 2016. Cependant, ces relevés sont coûteux à obtenir, et à partir de ceux déjà réalisés, vous voulez tenter de prédire les émissions de CO2 et la consommation totale d’énergie de bâtiments pour lesquels elles n’ont pas encore été mesurées.

Votre prédiction se basera sur les données déclaratives du permis d'exploitation commerciale (taille et usage des bâtiments, mention de travaux récents, date de construction..)

Vous cherchez également à évaluer l’intérêt de l’"ENERGY STAR Score" pour la prédiction d’émissions, qui est fastidieux à calculer avec l’approche utilisée actuellement par votre équipe.</p>

##### Votre mission

Vous sortez tout juste d’une réunion de brief avec votre équipe. Voici un récapitulatif de votre mission :

- Réaliser une courte analyse exploratoire.
- Tester différents modèles de prédiction afin de répondre au mieux à la problématique.

Avant de quitter la salle de brief, Douglas, le project lead, vous donne quelques pistes, et erreurs à éviter :

>L’objectif est de te passer des relevés de consommation annuels (attention à la fuite de données), mais rien ne t'interdit d’en déduire des variables plus simples (nature et proportions des sources d’énergie utilisées). 
>
>Fais bien attention au traitement des différentes variables, à la fois pour trouver de nouvelles informations (peut-on déduire des choses intéressantes d’une simple adresse ?) et optimiser les performances en appliquant des transformations simples aux variables (normalisation, passage au log, etc.).
>
>Mets en place une évaluation rigoureuse des performances de la régression, et optimise les hyperparamètres et le choix d’algorithme de ML à l’aide d’une validation croisée.
>

#### Let's go :

In [1]:
###################################################################################################
####################                       Import librairies                    ###################
###################################################################################################
import pandas as pd
import numpy as np
from numpy import isnan
from global_functions_variables import *


center_seattle_loc = {"latitude" : 47.6062095, "longitude" : -122.3320708}

useless_variables = ['SPD Beats', #no idea what it means but no related to energy
                     'City Council Districts', #93% of missing values
                     'OtherFuelUse(kBtu)', # only 17 actual values
                     '2010 Census Tracts', #93% of missing values
                     'Outlier', #97% of missing values
                     'PropertyGFATotal' #Already have parking and building, use it to check errors
                    ]
variables_test = ['Seattle Police Department Micro Community Policing Plan Areas',
                  'YearsENERGYSTARCertified',
                  'ENERGYSTARScore'
                 ]

variables_train = []

#### Les fichiers CSV :

<p>Les relevés énergétiques sont stockés dans 2 fichiers csv :</p>

- Les relevés de 2015
- Les relevés de 2016

<p>On remarque rapidement que les deux bases de données n'ont pas le même nombre de colonnes.
Une analyse des différences nous apprend :</p>

1. Les variables de localisation du fichier de 2016 :<br>
    *Latitude*,<br>
    *Longitude*,<br>
    *City*,<br>
    *Address*,<br>
    *zipCode*,<br>
    *State*<br> sont **compressées** dans une seule variable *Location* dans le fichier de 2015

2. Les variables du fichier 2015 :<br> 
     *OtherFuelUse(kBtu)*,<br> 
     *2010 Census Tracts*,<br> 
     *Seattle Police Department Micro Community Policing Plan Areas*,<br> 
     *City Council Districts*,<br> 
     *SPD Beats*,<br>
     *Zip Codes*,<br>non présentes en 2016, sont **vides, faussées ou sans rapport avec la consommation d'énergie**
    
3. Les dernières différences de variables représentent **les mêmes données sous la même métrique** mais **sous un autre nom**.

<p>Pour rassembler ces deux bases de données il faut donc :</p>

1. Décompresser la variable *Location* de 2015 et sauvegarder ses valeurs dans de nouvelles colonnes de même noms que la version 2016

2. Retirer les colonnes non pertinentes du DataFrame de 2015

3. Renommer les variables restantes pour s'adapter aux variables de 2016

Une fois ces étapes effectuées, les deux DataFrames pourront être combinés afin de traiter les données de 2015 et de 2016 simultanément.

In [52]:
###################################################################################################
####################          Load and merge CSV files from 2015 and 2016       ###################
###################################################################################################

#read csv file and stock it in dataframe
data_2015 = pd.read_csv(r'Data/2015-building-energy-benchmarking.csv',
                        sep=',', encoding='utf-8', na_values='')
data_2016 = pd.read_csv(r'Data/2016-building-energy-benchmarking.csv',
                        sep=',', encoding='utf-8', na_values='')
print(f"The 2016 CSV file had {data_2016.shape[1]}\
 fields and the 2015 CSV file had {data_2015.shape[1]} fields")
    
# 1.Re-Organize location variables - Dezip latitude, longitude ect..
decompressed = {'Latitude': [],
                'Longitude' : [],
                'City' : [],
                'Address' : [],
                'ZipCode' : [],
                'State': []}

for loc in data_2015['Location']:
    loc_dict = eval(loc)
    address_dict = eval(loc_dict['human_address'])
    decompressed['Latitude'].append(loc_dict['latitude'])
    decompressed['Longitude'].append(loc_dict['longitude'])
    decompressed['City'].append(address_dict['city'])
    decompressed['Address'].append(address_dict['address'])
    decompressed['ZipCode'].append(address_dict['zip'])
    decompressed['State'].append(address_dict['state'])
data_2015 = pd.concat([data_2015.drop('Location',axis=1), pd.DataFrame(decompressed)],axis=1)

# 2. Remove variables only present in 2015 AND empty, irrelevant or wrong :
data_2015.drop(["OtherFuelUse(kBtu)",'2010 Census Tracts',
                'Seattle Police Department Micro Community Policing Plan Areas',
                'City Council Districts',"SPD Beats", 'Zip Codes'], axis=1, inplace=True)

# For "GHGEmissions(MetricTonsCO2e) --> TotalGHGEmissions 
#we have checked beforehand that 2015 and 2016 dataset use the same metrics !

# 3. Rename 2015 variable to their 2016 names
data_2015.rename(columns={"Comment" : "Comments",
                          "GHGEmissions(MetricTonsCO2e)" : "TotalGHGEmissions",
                         "GHGEmissionsIntensity(kgCO2e/ft2)" : "GHGEmissionsIntensity"
                         },inplace=True)

# Finally, combine both into a 2015-2016 dataframe
if len(set(data_2016.columns).symmetric_difference(set(data_2015.columns))) == 0 :
    print("Both datasets have the same number of columns, it could be concatenate now")
    data_energy = pd.concat([data_2015, data_2016], axis=0)
else:
    print("Something gone wrong")
    
print(f"The final shape of 2015-2016 dataset is {data_energy.shape}")
data_energy['NumberofFloors'].unique()

The 2016 CSV file had 46 fields and the 2015 CSV file had 47 fields
Both datasets have the same number of columns, it could be concatenate now
The final shape of 2015-2016 dataset is (6716, 46)


array([12., 11., 41., 10., 18.,  2.,  8., 15.,  6., 25.,  9., 33., 28.,
        5., 19.,  7.,  1.,  3.,  4., 24., 20., 34.,  0., 16., 23., 17.,
       36., 22., 47., 29., 14., 49., 37., 42., 63., 13., 21., 55., 46.,
       30., 56., 26., 27., 76., 31., 99., 38., nan, 39., 32., 40.])

#### Première réduction de variables:

<p>Notre DataFrame possède 46 variables qu'il faut étudier pour déduire de leur pertinence et de leur impact sur notre projet. Pour cela, observer leur type, leur valeurs uniques, leur valeurs manquantes ou encore leur métrique apporte de précieuses informations.</p> 

##### Les variables de localisation:

<p>Les données de localisation sont importantes pour visualiser nos résultats sur une carte, ou étudier la consommation par quartier. L'emplacement d'un bâtiment est-il corrélé à sa consommation énergétique? Pour le découvrir, il faut conserver ces données. </p>

- *Latitude* et *Longitude* :
Axe Nord-Sud et Ouest-Est des coordonnées terrestres, permet une localisation précise et pratiquement unique (Utilisable pour détecter les doublons). **à conservé**

- *Neighborhood*, *ZipCode* et *CouncilDistrictCode* :
Chacune de ces variables représente des zones géographiques définies, à première vue, elles ne semblent pas se recouper et apporte donc des informations différentes. Les valeurs de *Neighborhood* et *ZipCode* ne sont pas standardisées (type string et int cohabitant, orthographe alternative etc...). Attention au boites postales ! Regroupement sous une seule variable à envisager.
**à conservé et à standardisé**

- *Address* :
Représentation "humaine" de la position du bâtiment, ne sera pas utilisable par nos systèmes, d'autant plus que de meilleures informations de positionnement existent. **à retirer**

- *State* et *City* :
Ces variables n'ont qu'une seule valeur possible dans ce jeu de données. **à retirer**


##### Les variables de description du bâtiment:

<p>Rappel du projet :</p>

>Votre prédiction se basera sur les données déclaratives du permis d'exploitation commerciale (taille et usage des bâtiments, mention de travaux récents, date de construction..)

<p>Les informations relatives aux bâtiments sont les données de base de notre système, sur lesquels on s'appuieras pour prédire la consommation. La récupération de ces données est peu coûteuse.</p> 

- *OSEBuildingID* et *TaxParcelIdentificationNumber* et *PropertyName* :
Variables d'identification, identifient les bâtiments ( ou leur propriétaire dans le cas de *TaxParcelIdentificationNumber*) par un numéro ou un nom. Utiles pour manipuler les données et détecter les doublons **à conservé**

- *YearBuilt*, *NumberofBuildings*, *NumberofFloors* :
Variables descriptives du bâtiments.
**à conservé**

- *PropertyGFATotal*, *PropertyGFAParking*, *PropertyGFABuilding(s)* :
GFA signifie Gross Floor Area : la superficie au sol du bâtiment en incluant les murs extérieurs. Ces variables représentent la superficie au sol du bâtiment avec ou sans prendre en compte le parking. La formule *PropertyGFATotal* = *PropertyGFAParking*+*PropertyGFABuilding(s)* semble indiquer une redondance.
**redondance : choisir quelles variables conserver ou supprimer**

- *BuildingType*, *PrimaryPropertyType*, *ListOfAllPropertyUseTypes* et autres :
Type d'usage du bâtiment selon différentes granularités. Si le bâtiment a plus d'un seul type d'usage, la surface au sol allouée aux trois premiers types d'usage est disponible dans les variables *LargestPropertyUseTypeGFA*, *SecondLargestPropertyUseTypeGFA* et *ThirdLargestPropertyUseTypeGFA*. Cette méthode de stockage d'information provoque un fort taux de valeurs manquantes. De plus, une redondance est présente entre *ListOfAllPropertyUseTypes* et les trois variables *LargestPropertyUseType*, *SecondLargestPropertyUseType*, *ThirdLargestPropertyUseType*. Des modifications sur ces variables sont donc à prévoir.
**à conservé et à modifié**

##### Les variables énergetiques:

       'YearsENERGYSTARCertified',
       'ENERGYSTARScore', 'SiteEUI(kBtu/sf)', 'SiteEUIWN(kBtu/sf)',
       'SourceEUI(kBtu/sf)', 'SourceEUIWN(kBtu/sf)', 'SiteEnergyUse(kBtu)',
       'SiteEnergyUseWN(kBtu)', 'SteamUse(kBtu)', 'Electricity(kWh)',
       'Electricity(kBtu)', 'NaturalGas(therms)', 'NaturalGas(kBtu)',
       'TotalGHGEmissions', 'GHGEmissionsIntensity',
       
##### Autres variables :

       'DefaultData', 'Comments',
       'ComplianceStatus', 'Outlier', 'DataYear'

##### Conclusion de la réduction des variables :

#### Nettoyage des données :

##### Supprimer les bâtiments résidentiels:

<p>Dans l'intitulé du projet il est dit :</p>

>Pour atteindre son objectif de ville neutre en émissions de carbone en 2050, votre équipe s’intéresse de près aux émissions **des bâtiments non destinés à l’habitation.**

<p>Une première action sera donc de retirer tout les bâtiments résidentiels de notre DataFrame. <br>Pour cela nous étudions les valeurs présentes dans *BuildingType*, nous y découvrons 3 valeurs représentant des bâtiment résidentiels :</p>

 - Multifamily MR (5-9)
 - Multifamily LR (1-4)
 - Multifamily HR (10+)
 
<p>Tout individus ayant ces valeurs dans le champs *BuildingType* est donc retirer de notre DataFrame</p>

##### Supprimer les colonnes non pertinentes:

<p>Les variables *City* et *State* n'ont qu'une seule valeur possible</p>
1. Les variables de localisation du fichier de 2016 :<br>
    *Latitude*,<br>
    *Longitude*,<br>
    *City*,<br>
    *Address*,<br>
    *zipCode*,<br>
    *State*<br> sont **compressées** dans une seule variable *Location* dans le fichier de 2015

2. Les variables du fichier 2015 :<br> 
     *OtherFuelUse(kBtu)*,<br> 
     *2010 Census Tracts*,<br> 
     *Seattle Police Department Micro Community Policing Plan Areas*,<br> 
     *City Council Districts*,<br> 
     *SPD Beats*,<br>
     *Zip Codes*,<br>non présentes en 2016, sont **vides, faussées ou sans rapport avec la consommation d'énergie**
    
3. Les dernières différences de variables représentent **les mêmes données sous la même métrique** mais **sous un autre nom**.

<p>Pour rassembler ces deux bases de données il faut donc :</p>

1. Décompresser la variable *Location* de 2015 et sauvegarder ses valeurs dans de nouvelles colonnes de même noms que la version 2016

2. Retirer les colonnes non pertinentes du DataFrame de 2015

3. Renommer les variables restantes pour s'adapter aux variables de 2016

Une fois ces étapes effectuées, les deux DataFrames pourront être combinés afin de traiter les données de 2015 et de 2016 simultanément.

In [39]:
###################################################################################################
####################                   Traitement des données                   ###################
###################################################################################################

# Traitement des données 1 : Removing residential building from dataset
resi_buildings = ['Multifamily MR (5-9)', 'Multifamily LR (1-4)', 'Multifamily HR (10+)']
data_energy = data_energy.loc[~data_energy['BuildingType'].isin(resi_buildings)]


for column in data_energy.columns:
    if data_energy[column].nunique()<20:
        print('Colonne {}, valeurs uniques :\n{}\n'.format(column, data_energy[column].unique()))
    else:
        print('Colonne {}, {} valeurs uniques'.format(column, data_energy[column].nunique()))






Colonne OSEBuildingID, 1698 valeurs uniques
Colonne DataYear, valeurs uniques :
[2015 2016]

Colonne BuildingType, valeurs uniques :
['NonResidential' 'Nonresidential COS' 'SPS-District K-12' 'Campus'
 'Nonresidential WA']

Colonne PrimaryPropertyType, 30 valeurs uniques
Colonne PropertyName, 3204 valeurs uniques
Colonne TaxParcelIdentificationNumber, 1835 valeurs uniques
Colonne CouncilDistrictCode, valeurs uniques :
[7 3 2 4 5 6 1]

Colonne Neighborhood, valeurs uniques :
['DOWNTOWN' 'SOUTHEAST' 'NORTHEAST' 'EAST' 'CENTRAL' 'NORTH'
 'MAGNOLIA / QUEEN ANNE' 'LAKE UNION' 'GREATER DUWAMISH' 'BALLARD'
 'NORTHWEST' 'SOUTHWEST' 'DELRIDGE' 'Central' 'Ballard' 'North' 'Delridge'
 'Northwest' 'DELRIDGE NEIGHBORHOODS']

Colonne YearBuilt, 113 valeurs uniques
Colonne NumberofBuildings, valeurs uniques :
[  1.   7.  11.  16.   4.   3.  39.   2.  10.   6.   0.  27.  14.   9.
   5.  nan   8.  23. 111.]

Colonne NumberofFloors, 45 valeurs uniques
Colonne PropertyGFATotal, 1667 valeurs uniques
Colon

In [14]:
############################      Trouver les doublons       #################################

doublons_list = {'coords': [], 'adress' : [], 'name' : [], 'taxNumber' : [], 'BuildId' : []}
id_set = set()
generic_name = ["warehouse","office","apartment","home depot","retail","public storage"]
list_set = []
data_doublons = original_data_2015[["OSEBuildingID", "Location","TaxParcelIdentificationNumber","PropertyName"]]
print(data_doublons.shape)
for i, val in data_doublons.T.items():
    big_set = set()
    loc = eval(val['Location'])
    lat_long = coords(loc['latitude'], loc['longitude'])
    adresse = eval(loc['human_address'])
    taxnum = val["TaxParcelIdentificationNumber"]
    name = val["PropertyName"]
    buildingid = val["OSEBuildingID"]
    if lat_long in doublons_list['coords'] :
        big_set.add(i)
        big_set.add(doublons_list['coords'].index(lat_long))
    if adresse in doublons_list['adress'] :
        big_set.add(i)
        big_set.add(doublons_list['adress'].index(adresse))
    if name in doublons_list['name']:
        big_set.add(i)
        big_set.add(doublons_list['name'].index(name))
    if taxnum in doublons_list['taxNumber']:
        big_set.add(i)
        big_set.add(doublons_list['taxNumber'].index(taxnum))
    if buildingid in doublons_list['BuildId']:
        big_set.add(i)
        big_set.add(doublons_list['BuildId'].index(buildingid))
    doublons_list['coords'].append(lat_long)
    doublons_list['adress'].append(adresse)
    doublons_list['name'].append(name)
    doublons_list['taxNumber'].append(taxnum)
    doublons_list['BuildId'].append(buildingid)
    if len(list_set) == 0 and len(big_set) != 0 :
        list_set.append(big_set)
    else :
        for x in list_set.copy():
            if big_set.intersection(x):
                #if some elements are intercorrelated, merge the group
                list_set[list_set.index(x)] = set.union(x, big_set)
                big_set = set() # avoid adding duplicate
                break
        if len(big_set) != 0 :
            # add a new correlated group only if no intersection 
            list_set.append(big_set)
            
for s in list_set:   
    data_doublons = original_data_2015[["OSEBuildingID", "Location","TaxParcelIdentificationNumber","PropertyName"]].iloc[list(s)]
    print(data_doublons.shape[0])
    print(data_doublons[["TaxParcelIdentificationNumber","PropertyName"]])

NameError: name 'original_data_2015' is not defined